<a href="https://colab.research.google.com/github/nurfnick/Data_Viz/blob/main/19_Missing_and_Incomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Missing and Incomplete

Often datasets will be missing entries.  There are many approaches we can take to dealing with these errors and omissions.  I will examine a dataset on the characters from <ins> The Lord of The Rings</ins>

## Finding NaN's

In [1]:
import pandas as pa

df = pa.read_csv('https://raw.githubusercontent.com/nurfnick/Data_Viz/main/lotr_characters.csv')

df.head()

,birth,death,gender,hair,height,name,race,realm,spouse
0,NaN,NaN,Female,NaN,NaN,Adanel,Men,NaN,Belemir
1,TA 2978,"February 26 ,3019",Male,Dark (book) Light brown (movie),NaN,Boromir,Men,NaN,NaN
2,NaN,"March ,3019",Male,NaN,NaN,Lagduf,Orcs,NaN,NaN
3,TA 280,TA 515,Male,NaN,NaN,Tarcil,Men,Arnor,Unnamed wife
4,NaN,NaN,Male,NaN,NaN,Fire-drake of Gondolin,Dragon,NaN,NaN


We see right away that there are lots of `NaN`'s.  This is an empty field in our dataset.  Some characters are mentioned but never given much more background than a name.

In [ ]:
df.isnull().sum(axis = 0)

birth     207
death     315
gender    143
hair      734
height    813
name        0
race      140
realm     714
spouse    403
dtype: int64

There are null values in every column except name.

In [ ]:
df.isnull().sum(axis = 1).value_counts().sort_index()

0     15
1     59
2    185
3    236
4    178
5     81
6     20
7      1
8    136
dtype: int64

Here we see that there are only 15 entries with all fields and 136 that are name only (since name was never blank!)  Let's look at just those characters.

In [ ]:
df[~df.isnull().any(axis = 1)]

,birth,death,gender,hair,height,name,race,realm,spouse
125,SA 3209,TA 2,Male,Black,Very tall almost 7'1,Isildur,Men,"Arnor,Gondor",Unnamed wife
134,"YT, and perhaps firstborn",Still Alive,Male,Probably Golden,Tall,Ingwë,Elves,"Valinor,Taniquetil",Unnamed wife
166,YT,FA 400,Male,Dark,Tall,Eöl,Elves,Nan Elmoth,Aredhel
186,TA 2990,FO 63,Male,Dirty blond,Tall-6'6,omer,Men,Rohan,Lothíriel after the War of the Ring
194,FA 532,"Still alive; departed to ,Aman, on ,September ...",Male,Dark,Tall,Elrond,Half-elven,Rivendell,Celebrían
204,SA 3119,SA 3441,Male,Brown,"7' 10""",Elendil,Men,"Arnor,Gondor",Unnamed wife
530,YT,"Still alive, departed over the sea in the earl...",Male,Silver,Tall,Celeborn,Elves,"Eregion,Lothlórien,Caras Galadhon",Galadriel
551,Possibly pre First Age,Unknown; possibly still alive,Most likely male,None,Huge,Watcher in the Water,Urulóki,Doors of Durin,Most likely none
579,3019,February 293019,Male,Dark (movie),"6' 6"" (movie)",Uglúk,Uruk-hai,Isengard,None
620,TA 2925,TA 3007,Male,Brown (film),"1.76m / 5'9"" (film)",Bain,Men,Dale,Unnamed wife


Of course we could ask for just the ones with 8 null values.

In [ ]:
df[df.isnull().sum(axis = 1) == 8].name

8                          Angrim
14                      Angelimar
17      Linda (Baggins) Proudfoot
18                 Bodo Proudfoot
40     Tanta (Hornblower) Baggins
                  ...            
886                        Andvír
891                        Amlach
904                         Aghan
905                       Agathor
907                      Aerandir
Name: name, Length: 136, dtype: object

I only included the names since the rest of the dataset was null!

Of course we can use this method to include only entries that have 4 or less null entries.

In [ ]:
df[df.isnull().sum(axis = 1) <= 4]

,birth,death,gender,hair,height,name,race,realm,spouse
1,TA 2978,"February 26 ,3019",Male,Dark (book) Light brown (movie),NaN,Boromir,Men,NaN,NaN
3,TA 280,TA 515,Male,NaN,NaN,Tarcil,Men,Arnor,Unnamed wife
5,SA 2709,SA 2962,Male,NaN,NaN,Ar-Adûnakhôr,Men,Númenor,Unnamed wife
7,YT,FA 455,Male,Golden,NaN,Angrod,Elves,NaN,Eldalótë
9,SA 3219,SA 3440,Male,NaN,NaN,Anárion,Men,Gondor,Unnamed wife
...,...,...,...,...,...,...,...,...,...
903,TA 2827,TA 2932,Male,NaN,NaN,Aglahad,Men,NaN,Unnamed wife
906,"Mid ,First Age",FA 495,Female,NaN,NaN,Aerin,Men,NaN,Brodda
908,"YT during the ,Noontide of Valinor",FA 455,Male,Golden,NaN,Aegnor,Elves,NaN,"Loved ,Andreth but remained unmarried"
909,TA 2917,TA 3010,Male,NaN,NaN,Adrahil II,Men,NaN,Unnamed wife


Maybe we only want the characters whose *realm* has been included.  We'll negate the `isnull()` command.

In [ ]:
df[~df.realm.isnull()]

,birth,death,gender,hair,height,name,race,realm,spouse
3,TA 280,TA 515,Male,NaN,NaN,Tarcil,Men,Arnor,Unnamed wife
5,SA 2709,SA 2962,Male,NaN,NaN,Ar-Adûnakhôr,Men,Númenor,Unnamed wife
9,SA 3219,SA 3440,Male,NaN,NaN,Anárion,Men,Gondor,Unnamed wife
10,SA 3118,Still alive,Male,NaN,Tall,Ar-Pharazôn,Men,Númenor,Tar-Míriel
11,SA 2876,SA 3102,Male,NaN,NaN,Ar-Sakalthôr,Men,Númenor,Unnamed wife
...,...,...,...,...,...,...,...,...,...
890,TA 726,TA 946,Male,NaN,NaN,Amlaith,Men,Arthedain,Unnamed wife
892,"Sometime during ,Years of the Trees, or the ,F...",SA 3434,Male,NaN,NaN,Amdír,Elves,Lórien,Unnamed wife
898,NaN,NaN,Female,NaN,NaN,Almarian,Men,Númenor,Tar-Meneldur
900,TA 2544,TA 2645,Male,NaN,NaN,Aldor,Men,Rohan,Unnamed wife


## Imputing

The simplest method for filling in `NaN`s is to just place a value there.

In [ ]:
df.fillna(value = 0)

,birth,death,gender,hair,height,name,race,realm,spouse
0,0,0,Female,0,0,Adanel,Men,0,Belemir
1,TA 2978,"February 26 ,3019",Male,Dark (book) Light brown (movie),0,Boromir,Men,0,0
2,0,"March ,3019",Male,0,0,Lagduf,Orcs,0,0
3,TA 280,TA 515,Male,0,0,Tarcil,Men,Arnor,Unnamed wife
4,0,0,Male,0,0,Fire-drake of Gondolin,Dragon,0,0
...,...,...,...,...,...,...,...,...,...
906,"Mid ,First Age",FA 495,Female,0,0,Aerin,Men,0,Brodda
907,0,0,0,0,0,Aerandir,0,0,0
908,"YT during the ,Noontide of Valinor",FA 455,Male,Golden,0,Aegnor,Elves,0,"Loved ,Andreth but remained unmarried"
909,TA 2917,TA 3010,Male,0,0,Adrahil II,Men,0,Unnamed wife


You should note right away that some of these zeros make no sense.  You might be more careful with your zeros.

In [ ]:
df.height.fillna(value = 0)

0      0
1      0
2      0
3      0
4      0
      ..
906    0
907    0
908    0
909    0
910    0
Name: height, Length: 911, dtype: object

Or you might not want to skew the average so much.  You could assign the mean if the remaining values were numerical.  Unfortuantely these are mostly strings with little hope of converting to a numerical value.

In [ ]:
df.height[~df.height.isnull()]

10                                 Tall
19                                 Tall
20     Tallest of the Elves of Gondolin
41                                 Tall
74                    Large and immense
                     ...               
831                                 8'5
850                                Tall
853                                Tall
873                        198cm (6'6")
881               As tall as a mountain
Name: height, Length: 98, dtype: object

We can also fill the empties by grabbing other values around our missing.

In [ ]:
df.height.fillna(method= 'pad')

0                        NaN
1                        NaN
2                        NaN
3                        NaN
4                        NaN
               ...          
906    As tall as a mountain
907    As tall as a mountain
908    As tall as a mountain
909    As tall as a mountain
910    As tall as a mountain
Name: height, Length: 911, dtype: object

`pad` took the last value and filled it forward.  We can also go the otherway with `bfill`

In [ ]:
df.height.fillna(method= 'bfill')

0      Tall
1      Tall
2      Tall
3      Tall
4      Tall
       ... 
906     NaN
907     NaN
908     NaN
909     NaN
910     NaN
Name: height, Length: 911, dtype: object

## Imputing by Category

There is no quantitative data here so I actually have to work a little harder than I'd like.  If height was just a number you'd run some code like 

```
df.height.fillna(df.groupby('realm').height.transform('mean'))
```

To fill the NaNs with the mean from there group.  To deal with the categories I'll need to get the most frequent from each category first.

In [27]:
df.height.fillna(df.groupby('realm').height.transform(lambda s: s.mode()))

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
906    NaN
907    NaN
908    NaN
909    NaN
910    NaN
Name: height, Length: 911, dtype: object

This is showing be that the most common height by each realm is mostly NaN.  We'll need to get rid of all that to help this imputation.

In [22]:
dfrh = df[(~df.realm.isna())&(~df.height.isna())]

dfrh.head(50)

,birth,death,gender,hair,height,name,race,realm,spouse
10,SA 3118,Still alive,Male,NaN,Tall,Ar-Pharazôn,Men,Númenor,Tar-Míriel
41,TA 2922,NaN,Female,Dark,Tall,Morwen Steelsheen,Men,Rohan,Thengel
74,"Unknown, possibly during the ,First Age","Unknown, possibly TA 3019",Female,NaN,Large and immense,Shelob,Great Spiders,Torech Ungol,NaN
75,TA 2050,NaN,Male,NaN,7'1,Shagrat,Black Uruk,Cirith Ungol,NaN
125,SA 3209,TA 2,Male,Black,Very tall almost 7'1,Isildur,Men,"Arnor,Gondor",Unnamed wife
134,"YT, and perhaps firstborn",Still Alive,Male,Probably Golden,Tall,Ingwë,Elves,"Valinor,Taniquetil",Unnamed wife
137,NaN,Still Alive,Female,Golden,Tall,Indis,Elves,Tirion,Finwë
143,NaN,NaN,Male,"Varies, White",Varies,Eru Ilúvatar,God,Timeless Halls,None
166,YT,FA 400,Male,Dark,Tall,Eöl,Elves,Nan Elmoth,Aredhel
186,TA 2990,FO 63,Male,Dirty blond,Tall-6'6,omer,Men,Rohan,Lothíriel after the War of the Ring


In [35]:
dfrh.groupby(['realm']).height.transform(lambda s: s.mode().iat[0])

10                                Tall
41                                Tall
74                   Large and immense
75                                 7'1
125                             7' 10"
134                               Tall
137                               Tall
143                             Varies
166                               Tall
186                               Tall
194                               Tall
204                             7' 10"
295               4'5 - 5' (Estimate) 
334                             Varies
399                             Varies
426                 Any size by choice
430                        Tall (6'4")
462                 5'9" - 6'4" (film)
469                                7'1
497                               Tall
530                               Tall
532                           Gigantic
545                7' 1" (2.13 metres)
551                               Huge
579                      6' 6" (movie)
609                      

In [46]:
changeheight = df.height.fillna(dfrh.groupby('realm').height.transform(lambda s: s.mode()))

df[(changeheight != df.height) & ~changeheight.isnull()]

,birth,death,gender,hair,height,name,race,realm,spouse


In [39]:
df.loc[:,['height','realm']]

,height,realm
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,Arnor
4,NaN,NaN
...,...,...
906,NaN,NaN
907,NaN,NaN
908,NaN,NaN
909,NaN,NaN


In [42]:
dfrh.groupby('realm').height.mode()

AttributeError: ignored